In [1]:
from KmeansPPcenters import KMeanPlusPlus
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
import random
import scipy.io as scio
from numpy import genfromtxt
from KMeansOut import kmeansOutliers, cost
from sklearn.metrics import average_precision_score, precision_recall_curve
from Noise import add_random_noise, compute_phi_star, cost
from sklearn import cluster, datasets, mixture
import pandas as pd

In [2]:
def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.randint(start, end)) 
  
    return res 

In [7]:
susy_data = pd.read_csv('realData/SUSY/SUSY.csv', header=None)
labels= susy_data.values[:,0:1]
processed_data= np.delete(susy_data.values,0, axis=1)


In [8]:
data_all= processed_data
data_part_8= processed_data[0:10000,0:8]
data_part_10= processed_data[:,0:10]

In [9]:
data= data_part_8

In [10]:
num_clusters=[5]
zs =[25, 50, 100]
min_values=[-16, 0, 7.693475e-08]
max_values= [15, 20, 33]
tol= .05
itr=5

In [15]:
for num_cluster in num_clusters:
    for z in zs:
        for min_value in min_values:
            for max_value in max_values:
                print("num_cluster:{}, z:{}, min_value:{}, max_value:{}".format(num_cluster, z, min_value, max_value))
                #Step 1: Adding noise
                data_with_outliers, z_indx = add_random_noise(data, z, min_value, max_value)
                data_inliers= np.delete(data, z_indx, axis=0)
                
                # Step 2:Pick random centers
                random_indx= random.randint(0, len(data_with_outliers)-1)
                init=data_with_outliers[random_indx]
                                
                # Step 3 :Initialization for LLoyd compute random centers
                KPP=KMeanPlusPlus(num_clusters=num_cluster, init=init)
                KPP.fit(data_with_outliers)
                #centers= KPP.centers
                
                init_index= Rand(0, len(data_with_outliers), num_cluster)
                centers= data_with_outliers[init_index]
                
                new_centers=np.zeros((num_cluster, 8))
                #print("new_centers:{}".format(new_centers))
                #convergence of LLoyd
                
                for i in range(itr):
                    print(i)
                    
                    #centers= skin_data[np.random.choice(np.arange(len(data_with_outliers)-1), size= 10)]
                    #Step 4: Compute distance between all centers and points 
                    dist= distance.cdist(data_with_outliers, centers)
                    dist = np.amin(dist, axis = 1)
                    
                    #Step 5: Remove furthest z points
                    indx_list = np.argpartition(dist, -z)[-z:]
                    data_new = np.delete(data, indx_list, axis=0)
                    
                    #Step 6: Compute distance with data_new and centers & save the index of min distance in cid
                    dist_new = distance.cdist(data_new, centers)
                    cid = np.argmin(dist_new, axis=1)
                    dist_new = np.amin(dist_new, axis = 1)
                    
                    #Step 7: Compute new centers as mean of data points in each clusters
                    for j in range(num_cluster):
                        if len(data_new[cid==j])>0:
                            new_centers[j]= np.mean(data_new[cid==j], axis=0)
                            #print(new_centers[j])
                    #print("New centers now:{}".format(new_centers))
                    #new_centers= skin_data[np.random.choice(np.arange(len(data_with_outliers)-1), size= 10)]
                    
                    
                    
                    #Generate set them as new centers
                    old_centers= centers
                    centers= new_centers
                    
                    #Step 8: Check the difference in the set of centers
                    isOPTIMAL= True

                    if (len(np.setdiff1d(new_centers,old_centers)))> tol:
                        isOPTIMAL= False
                    
                    #Step 9: Stop if value less than tolerence
                    if isOPTIMAL:
                        break
                
                #Step 10: Compute precision and recall
                precision = len(np.intersect1d(z_indx, indx_list))/len(z_indx)
                recall = len(np.intersect1d(z_indx, indx_list))/len(indx_list)
                #x1= KPP.predict(data_with_outliers)
                print(("Precision:{}, recall:{}". format(precision, recall)))
                

num_cluster:5, z:25, min_value:-16, max_value:15
0
1
Precision:0.5, recall:0.5
num_cluster:5, z:25, min_value:-16, max_value:20
0
1
Precision:0.5, recall:0.5
num_cluster:5, z:25, min_value:-16, max_value:33
0
1
Precision:0.5102040816326531, recall:0.5102040816326531
num_cluster:5, z:25, min_value:0, max_value:15
0
1
Precision:0.5, recall:0.5
num_cluster:5, z:25, min_value:0, max_value:20
0
1
Precision:0.5208333333333334, recall:0.5208333333333334
num_cluster:5, z:25, min_value:0, max_value:33
0
1
Precision:0.5319148936170213, recall:0.5319148936170213
num_cluster:5, z:25, min_value:7.693475e-08, max_value:15
0
1
Precision:0.5, recall:0.5
num_cluster:5, z:25, min_value:7.693475e-08, max_value:20
0
1
Precision:0.5102040816326531, recall:0.5102040816326531
num_cluster:5, z:25, min_value:7.693475e-08, max_value:33
0
1
Precision:0.5434782608695652, recall:0.5434782608695652
num_cluster:5, z:50, min_value:-16, max_value:15
0
1
Precision:0.5050505050505051, recall:0.5050505050505051
num_clust